#Импортируем необходимые библиотеки для дальнейшей работы

In [2]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import statistics

from datetime import datetime

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import cross_val_score

# Скачиваем данные

In [3]:
orders = pd.read_csv("orders.csv")
delay = pd.read_csv("partners_delays.csv")
new_orders = pd.read_csv("new_orders.csv")

#Отделяем тот набор данных, относительно которых нам нужно будет предсказать кол-во курьеров и их delay_rate

In [4]:
X_test_global = new_orders.loc[~new_orders['date'].isin(orders['date'])]

# Делаем базовую обработку данных с датой и выбираем нужное для нас (день, время)

In [5]:
delay['dttm'] = pd.to_datetime(delay['dttm'])
delay['time'] = delay.dttm.dt.time
delay['time'] = [str(i) for i in delay['time']]
delay['day'] = delay.dttm.dt.day
delay['day'] = [str(i) for i in delay['day']]
delay.rename(columns={'dttm':'date'}, inplace=True)

orders['date'] = pd.to_datetime(orders['date'])
orders['time'] = orders.date.dt.time
orders['time'] = [str(i) for i in orders['time']]
orders['day'] = orders.date.dt.day
orders['day'] = [str(i) for i in orders['day']]
X_test_global['date'] = pd.to_datetime(X_test_global['date'])
X_test_global['time'] = X_test_global.date.dt.time

delay.drop(delay[((delay == '09:00:00') | (delay == '22:00:00')).any(axis=1)].index, inplace=True)  #удаляем 0-вой час работы и после-последний час работы

delay['day'] = delay.date.dt.day
delay['time'] = delay.date.dt.time
orders['day'] = orders.date.dt.day
orders['time'] = orders.date.dt.time

#Соединяем все наши данные в один DataFrame

In [7]:
merge_table = delay.merge(orders, how='left', on=['delivery_area_id','date', 'time', 'day'])
merge_table.dropna(inplace=True)

#Переводим время в числовой тип, чтобы потом его можно было использовать в обучении модели

In [8]:
merge_table['int_time'] = [int(i.strftime("%H%M%S")) for i in merge_table.time]
merge_table.drop(columns=['time'], inplace=True)
X_test_global['int_time'] = [int(i.strftime("%H%M%S")) for i in X_test_global.date]

#Импортируем дополнительные библиотеки

In [9]:
from sklearn.preprocessing import StandardScaler
scaller = StandardScaler

# Обучаем модель на имеющихся данных, путем разбивки на train и test мн-ва. Смотрим на точность нашего алгоритма

In [12]:
X = merge_table.iloc[:,[0,5,6]]
y = merge_table.iloc[:,2]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train_scal = StandardScaler().fit_transform(X_train)
X_test_scal = StandardScaler().fit_transform(X_test)

params = {'n_estimators':200,
          'max_depth':25,
          'criterion':'mse',
          'learning_rate':0.03,
          'min_samples_leaf':16,
          'min_samples_split':16
          }
model_xgb_for_partners_cnt = XGBClassifier(**params)
model_xgb_for_partners_cnt.fit(X_train_scal, y_train)

y_pred = model_xgb_for_partners_cnt.predict(X_test_scal)
print('xgb_for_partners_cnt =',mean_squared_error(y_test,y_pred))  #0.77

xgb_for_partners_cnt = 0.771771143758387


#Смотрим на точность нашей обученной модели

In [14]:
y_pred_after = [int(i) for i in y_pred]
y_test_after = [int(i) for i in y_test]
accuracy_score(y_pred_after, y_test_after)  #73.9%

0.7390599675850892

In [15]:
X_test_global

,delivery_area_id,date,orders_cnt,time,int_time
2839,0,2021-12-01 10:00:00,21.0,10:00:00,100000
2840,0,2021-12-01 11:00:00,10.0,11:00:00,110000
2841,0,2021-12-01 12:00:00,11.0,12:00:00,120000
2842,0,2021-12-01 13:00:00,11.0,13:00:00,130000
2843,0,2021-12-01 14:00:00,12.0,14:00:00,140000
...,...,...,...,...,...
934904,592,2021-12-07 17:00:00,0.0,17:00:00,170000
934905,592,2021-12-07 18:00:00,0.0,18:00:00,180000
934906,592,2021-12-07 19:00:00,0.0,19:00:00,190000
934907,592,2021-12-07 20:00:00,-0.0,20:00:00,200000


# Даем предикт по новому числу курьеров для новых заказов на несколько дней вперед

In [16]:
X_test_global_for_predicting = X_test_global.iloc[:,[0,2,4]]
X_test_global_for_predicting_scal = StandardScaler().fit_transform(X_test_global_for_predicting)
y_pred_global = model_xgb_for_partners_cnt.predict(X_test_global_for_predicting_scal)

array([[-1.73038545,  2.55147295, -1.59221267],
       [-1.73038545,  0.70499094, -1.30234938],
       [-1.73038545,  0.87285294, -1.01248608],
       ...,
       [ 1.71876811, -0.97362908,  1.01655699],
       [ 1.71876811, -0.97362908,  1.30642029],
       [ 1.71876811, -0.97362908,  1.59628359]])

In [18]:
#len(y_pred_global) == len(X_test_global_for_predicting_scal)

48988

#Готовим данные для предсказания delay_rate

In [20]:
X_test_global['partners_cnt'] = y_pred_global
X_test_global['orders_per_human'] = X_test_global.orders_cnt / X_test_global.partners_cnt
merge_table['orders_per_human'] = merge_table.orders_cnt / merge_table.partners_cnt
merge_table['for_pred_delay'] = merge_table.delay_rate*100
#X_test_global.to_csv('1st_step_excel.csv',index=True)

#Готовим delay_rate для предсказания модели

In [30]:
merge_table['for_pred_delay'] = [int(i*100) for i in merge_table.delay_rate]
merge_table

,delivery_area_id,date,partners_cnt,delay_rate,day,orders_cnt,int_time,orders_per_human,for_pred_delay
0,0,2021-04-01 10:00:00,4.0,0.111111,1,9.0,100000,2.25,11
1,0,2021-04-01 11:00:00,4.0,0.000000,1,1.0,110000,0.25,0
3,0,2021-04-01 13:00:00,1.0,0.000000,1,1.0,130000,1.00,0
4,0,2021-04-01 14:00:00,1.0,0.000000,1,1.0,140000,1.00,0
5,0,2021-04-01 15:00:00,1.0,0.000000,1,2.0,150000,2.00,0
...,...,...,...,...,...,...,...,...,...
1194043,592,2021-11-30 16:00:00,3.0,0.000000,30,6.0,160000,2.00,0
1194044,592,2021-11-30 17:00:00,2.0,0.000000,30,3.0,170000,1.50,0
1194045,592,2021-11-30 18:00:00,2.0,0.000000,30,2.0,180000,1.00,0
1194046,592,2021-11-30 19:00:00,2.0,0.000000,30,4.0,190000,2.00,0


# Прогнозируем какой будет delay_rate с новым кол-вом курьеров (спронозируемым на предыдущем шаге). Для этого сначала обучаем модель на уже имеющихся данных

In [31]:
params = {'n_estimators':200,
          'max_depth':25,
          'criterion':'mse',
          'learning_rate':0.03,
          'min_samples_leaf':16,
          'min_samples_split':16
          }

In [32]:
X_pred_delay = merge_table.iloc[:,[0,2,5,6,7]]
y_pred_delay = merge_table.iloc[:,8]
X_train_delay, X_test_delay, y_train_delay, y_test_delay = train_test_split(X_pred_delay, y_pred_delay, test_size=0.33, random_state=42)
X_train_delay_scale = StandardScaler().fit_transform(X_train_delay)
X_test_delay_scale = StandardScaler().fit_transform(X_test_delay)

model_xgb_for_delay_rate = XGBClassifier(**params)
model_xgb_for_delay_rate.fit(X_train_delay_scale, y_train_delay)

y_pred_delay = model_xgb_for_delay_rate.predict(X_test_delay_scale)
print('xgb =',mean_squared_error(y_test_delay,y_pred_delay))

xgb = 149.63665673306494


#Смотрим на точность предсказаний для обученной модели

In [33]:
y_pred_after_delay = [int(i) for i in y_pred_delay]
y_test_after_delay = [int(i) for i in y_test_delay]
accuracy_score(y_pred_after_delay, y_test_after_delay)  #83%

0.8259563270071975

#Средний delay_rate для тестовой модели (в %)

In [39]:
y_pred_delay.mean()  

1.0262525923215002

#Обучаем и смотрим на результат на новых курьерах и новых сменах

In [48]:
X_test_global_for_predicting_delay_rate = X_test_global.iloc[:,[0,2,4,5,6]]
X_test_global_for_predicting_delay_rate_scale = StandardScaler().fit_transform(X_test_global_for_predicting_delay_rate)
y_pred_global_delay = model_xgb_for_delay_rate.predict(X_test_global_for_predicting_delay_rate_scale)
y_pred_global_delay.mean() / 100  #0.8% - средний delay_rate для предсказанных курьеров

0.008270392749244713

#Так как мы умножали на 100 delay_rate для предсказания, возвращаем его к исходной величине, разделив на ту же величину (100)

In [46]:
X_test_global['delay_rate'] = y_pred_global_delay / 100
X_test_global

,delivery_area_id,date,orders_cnt,time,int_time,partners_cnt,orders_per_human,delay_rate
2839,0,2021-12-01 10:00:00,21.0,10:00:00,100000,5.0,4.200000,0.0
2840,0,2021-12-01 11:00:00,10.0,11:00:00,110000,6.0,1.666667,0.0
2841,0,2021-12-01 12:00:00,11.0,12:00:00,120000,5.0,2.200000,0.0
2842,0,2021-12-01 13:00:00,11.0,13:00:00,130000,3.0,3.666667,0.0
2843,0,2021-12-01 14:00:00,12.0,14:00:00,140000,3.0,4.000000,0.0
...,...,...,...,...,...,...,...,...
934904,592,2021-12-07 17:00:00,0.0,17:00:00,170000,1.0,0.000000,0.0
934905,592,2021-12-07 18:00:00,0.0,18:00:00,180000,1.0,0.000000,0.0
934906,592,2021-12-07 19:00:00,0.0,19:00:00,190000,1.0,0.000000,0.0
934907,592,2021-12-07 20:00:00,-0.0,20:00:00,200000,1.0,-0.000000,0.0


Собираем все данные в общий ответ (делаем новый файл с предиктами и старой информацией)

In [105]:
merge_table.drop(columns=['day','for_pred_delay'], inplace=True)
X_test_global.drop(columns=['time'], inplace=True)
all_end = pd.concat([merge_table, X_test_global], axis=0)
all_end.drop(columns=['orders_cnt', 'int_time', 'orders_per_human'], inplace=True)
all_end['int_time'] = [int(i.strftime("%Y%m%d%H%M%S")) for i in all_end.date]
all_end.sort_values(by=['delivery_area_id', 'int_time'], inplace=True)
all_end.drop(columns=['int_time'], inplace=True)
all_end.to_csv('new_partners_cnt.csv')